### Coding a sampling loop

Here, we want to write a sampling loop using the `Model` class. This will take us into the intrinsics of `tensorflow`, with its `Session` object that requires a `feed_dict` to evaluate nodes. Moreover, we will understand that `Model` is essentially just a bit init() function and a dictionary to get at the initiated nodes.

In [1]:
import sys
sys.path.insert(1, '/home/heber/packages/TATi-unstable/lib/python3.5/site-packages')


So, let's go through this a bit quicker.

#### Instantiate options

In [2]:
from TATi.options.pythonoptions import PythonOptions

options = PythonOptions()
options.set_options(
    batch_data_files=["dataset-twoclusters.csv"],
    fix_parameters="output/biases/Variable:0=-0.045677684",
    friction_constant=1.,
    inverse_temperature=10.,
    output_activation="linear",
    loss="mean_squared",
    max_steps=100,
    parse_parameters_file="training.csv",
    parse_steps=[100],
    sampler="BAOAB",
    seed=426,
    step_width=0.1)

#### Instantiate `Model`

In [3]:
from TATi.model import Model as tati
nn = tati(options)
nn.init_input_pipeline()
nn.init_network(None, setup="sample")

# reset dataset to set its "iterator" to start
nn.reset_dataset()

#### The loop

Before we create the loop, we need to fill a `feed_dict`. This is a dictionary with values for every `tensorflow.placeholder` node. As the name suggest, these are placeholders for values fed by the user.

In [4]:
feed_dict = {
    nn.state.nn[0].placeholder_nodes["friction_constant"]: options.friction_constant,
    nn.state.nn[0].placeholder_nodes["inverse_temperature"]: options.inverse_temperature,
    nn.state.nn[0].placeholder_nodes["step_width"]: options.step_width,
}

Moreover, we gather a list of nodes which we want to evaluate per step. Among them is `sample_step`, that triggers the a single update step as implemented by the *BAOAB* sampler.

In [5]:
step_nodes = [nn.state.nn[0].get(item) for item in ["sample_step", "accuracy", "global_step", "loss"]]

Next, we loop for `max_steps` steps: In the loop we evaluate a specific node,  For this, however, we need the feed_dict contain the batch's features and labels.


In [6]:
for i in range(options.max_steps):
    # place the batch inside feed_dict
    features, labels = nn.state.input_pipeline.next_batch(nn.sess)
    feed_dict.update({
        nn.state.xinput: features,
        nn.state.true_labels: labels
    })
    
    # perform the sampling step
    _, acc, step, loss = nn.sess.run(step_nodes, feed_dict=feed_dict)
    
    # print loss
    print(step, loss)

101 0.021974392
102 0.022016129
103 0.022010544
104 0.022782605
105 0.025476754
106 0.030133966
107 0.033590313
108 0.029148709
109 0.022416748
110 0.031302772
111 0.06415214
112 0.096364304
113 0.113083005
114 0.10122893
115 0.074376866
116 0.045133177
117 0.025384735
118 0.028010163
119 0.045658525
120 0.067148186
121 0.08312476
122 0.09350805
123 0.095763855
124 0.102452435
125 0.09386392
126 0.06980161
127 0.056621633
128 0.053400915
129 0.05912881
130 0.06566224
131 0.05265801
132 0.037870813
133 0.03325355
134 0.045062046
135 0.04695704
136 0.03966055
137 0.03314541
138 0.025108367
139 0.025417764
140 0.033234216
141 0.039734192
142 0.044298425
143 0.06495814
144 0.08764149
145 0.090220496
146 0.06244477
147 0.02853256
148 0.028489823
149 0.06348887
150 0.0916963
151 0.07728517
152 0.043520056
153 0.02381428
154 0.034122657
155 0.09346721
156 0.23239116
157 0.34423405
158 0.3769963
159 0.3399719
160 0.24748191
161 0.19779679
162 0.14594564
163 0.07958278
164 0.05171747
165 0.1004

### Summary

- `Model` captures its state in the `ModelState`. In that state there is an instance of `NeuralNetwork`, one per walker.
- sampling is triggered by evaluating a specific node `sample_step`.
- evaluation of nodes requires a `Session` object and for each required `tensorflow.placeholder` an entry in a so-called `feed_dict`.